# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
pip install -U numpy==1.18.5

  Using cached numpy-1.18.5-cp38-cp38-manylinux1_x86_64.whl (20.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.2
    Uninstalling numpy-1.20.2:
      Successfully uninstalled numpy-1.20.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.4.1 requires gast==0.3.3, but you have gast 0.4.0 which is incompatible.
tensorflow 2.4.1 requires grpcio~=1.32.0, but you have grpcio 1.37.1 which is incompatible.
tensorflow 2.4.1 requires numpy~=1.19.2, but you have numpy 1.18.5 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import os
import datetime
from PIL import Image
def imread(path):
    from PIL import Image
    return np.array(Image.open(path))

def imresize(img, size):
    from PIL import Image
    return np.array(Image.fromarray(img).resize(size))

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 20

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
x = 18 # number of frames
y = 120 # image width
z = 120 # image height
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx =[0,1,2,4,6,8,10,12,14,16,18,20,22,24,26,27,28,29] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches =len(t)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image=np.array(Image.fromarray((image * 255).astype(np.uint8)).resize((120, 120)).convert('RGB'))                    
                    batch_data[folder,idx,:,:,0] =  (image[:,:,0])/255#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] =  (image[:,:,1])/255#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] =  (image[:,:,2])/255#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if(len(t)%batch_size)!=0:
            batch_data = np.zeros((len(t)%batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((len(t)%batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(len(t)%batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image=np.array(Image.fromarray((image * 255).astype(np.uint8)).resize((120, 120)).convert('RGB'))
                    batch_data[folder,idx,:,:,0] = (image[:,:,0])/255#normalise and feed in the image # divide by 255.0
                    batch_data[folder,idx,:,:,1] = (image[:,:,1])/255#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2])/255#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1 # OHE
            yield batch_data, batch_labels


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [5]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30  # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


## CNN3D

## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [6]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout,GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from tensorflow.keras.layers import Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers

#write your model here
model = Sequential()
model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,120,120,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))

model.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,120,120,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,120,120,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [7]:
optimiser =  optimizers.Adam(lr=0.001) # write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 18, 120, 120, 64)  5248      
_________________________________________________________________
batch_normalization (BatchNo (None, 18, 120, 120, 64)  256       
_________________________________________________________________
activation (Activation)      (None, 18, 120, 120, 64)  0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 9, 60, 120, 64)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 9, 60, 120, 128)   221312    
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 60, 120, 128)   512       
_________________________________________________________________
activation_1 (Activation)    (None, 9, 60, 120, 128)   0

In [8]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [9]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_weights_only=False, mode='auto', save_freq='epoch')

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [10]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [11]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 20
Epoch 1/30
34/34 [==============================] - 136s 4s/step - loss: 306.5118 - categorical_accuracy: 0.2733 - val_loss: 82.3638 - val_categorical_accuracy: 0.1600
Epoch 2/30
34/34 [==============================] - 51s 2s/step - loss: 51.1940 - categorical_accuracy: 0.4178 - val_loss: 6.1388 - val_categorical_accuracy: 0.2000
Epoch 3/30
34/34 [==============================] - 50s 1s/step - loss: 7.1393 - categorical_accuracy: 0.4213 - val_loss: 2.9288 - val_categorical_accuracy: 0.2500
Epoch 4/30
34/34 [==============================] - 50s 1s/step - loss: 3.1149 - categorical_accuracy: 0.3855 - val_loss: 2.0094 - val_categorical_accuracy: 0.2100
Epoch 5/30
34/34 [==============================] - 50s 1s/step - loss: 1.4825 - categorical_accuracy: 0.3728 - val_loss: 3.4218 - val_categorical_accuracy: 0.1900
Epoch 6/30
34/34 [==============================] - 51s 1s/step - loss: 1.4715 - categorical_accuracy: 0.3512 - val_loss: 5

#### Comment

Accuracy for both training and validation data is very low.

Next, let's change x,y and z and reduce the number of epochs to 20

### Model 2 :  Experiment Number 1 

In [11]:
x = 21 # number of frames
y = 84 # image width
z = 84 # image height
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx =[0,1,2,3,4,6,8,10,12,14,15,16,18,20,22,24,25,26,27,28,29] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches =len(t)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image=np.array(Image.fromarray((image * 255).astype(np.uint8)).resize((84, 84)).convert('RGB'))                    
                    batch_data[folder,idx,:,:,0] =  (image[:,:,0])/255#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] =  (image[:,:,1])/255#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] =  (image[:,:,2])/255#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if(len(t)%batch_size)!=0:
            batch_data = np.zeros((len(t)%batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((len(t)%batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(len(t)%batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image=np.array(Image.fromarray((image * 255).astype(np.uint8)).resize((84, 84)).convert('RGB'))
                    batch_data[folder,idx,:,:,0] = (image[:,:,0])/255#normalise and feed in the image # divide by 255.0
                    batch_data[folder,idx,:,:,1] = (image[:,:,1])/255#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2])/255#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1 # OHE
            yield batch_data, batch_labels


In [12]:
input_shape = (21,84,84,3)

#write your model here
model_2 = Sequential()
model_2.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=input_shape))
model_2.add(BatchNormalization())
model_2.add(Activation('relu'))
model_2.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))

model_2.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same', input_shape=input_shape))
model_2.add(BatchNormalization())
model_2.add(Activation('relu'))
model_2.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model_2.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', input_shape=input_shape))
model_2.add(BatchNormalization())
model_2.add(Activation('relu'))
model_2.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))

model_2.add(Conv3D(512, (3,3,3), strides=(1,1,1), padding='same', input_shape=input_shape))
model_2.add(BatchNormalization())
model_2.add(Activation('relu'))
model_2.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))

#Flatten Layers
model_2.add(Flatten())

model_2.add(Dense(1000, activation='relu'))
model_2.add(Dropout(0.5))

model_2.add(Dense(500, activation='relu'))
model_2.add(Dropout(0.5))

#softmax layer
model_2.add(Dense(5, activation='softmax'))

In [13]:
optimiser =  optimizers.Adam(lr=0.001) # write your optimizer
model_2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_2.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_3 (Conv3D)            (None, 21, 84, 84, 64)    5248      
_________________________________________________________________
batch_normalization_3 (Batch (None, 21, 84, 84, 64)    256       
_________________________________________________________________
activation_3 (Activation)    (None, 21, 84, 84, 64)    0         
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 10, 42, 84, 64)    0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 10, 42, 84, 128)   221312    
_________________________________________________________________
batch_normalization_4 (Batch (None, 10, 42, 84, 128)   512       
_________________________________________________________________
activation_4 (Activation)    (None, 10, 42, 84, 128)  

In [14]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [16]:
model_2.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=20, verbose=1, 
                    validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 20
Epoch 1/20
34/34 [==============================] - 145s 4s/step - loss: 101.2555 - categorical_accuracy: 0.2433 - val_loss: 16.8821 - val_categorical_accuracy: 0.2400
Epoch 2/20
34/34 [==============================] - 49s 1s/step - loss: 22.2616 - categorical_accuracy: 0.3000 - val_loss: 2.4095 - val_categorical_accuracy: 0.1700
Epoch 3/20
34/34 [==============================] - 48s 1s/step - loss: 9.6193 - categorical_accuracy: 0.3004 - val_loss: 1.9910 - val_categorical_accuracy: 0.1800
Epoch 4/20
34/34 [==============================] - 48s 1s/step - loss: 7.7487 - categorical_accuracy: 0.2877 - val_loss: 2.0298 - val_categorical_accuracy: 0.1200
Epoch 5/20
34/34 [==============================] - 49s 1s/step - loss: 5.1722 - categorical_accuracy: 0.3552 - val_loss: 1.7248 - val_categorical_accuracy: 0.2200
Epoch 6/20
34/34 [==============================] - 48s 1s/step - loss: 3.1766 - categorical_accuracy: 0.3088 - val_loss: 1

##### Comment:

Performance has dropped from previous model. We will retain the image size as 120x120 and reduce the filter size


### Model 3 :  Experiment Number 2
- Keep image size as 120x120 and reduce the filter size

In [15]:
x = 21 # number of frames
y = 120 # image width
z = 120 # image height
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx =[0,1,2,3,4,6,8,10,12,14,15,16,18,20,22,24,25,26,27,28,29] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches =len(t)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image=np.array(Image.fromarray((image * 255).astype(np.uint8)).resize((120, 120)).convert('RGB'))                    
                    batch_data[folder,idx,:,:,0] =  (image[:,:,0])/255#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] =  (image[:,:,1])/255#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] =  (image[:,:,2])/255#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if(len(t)%batch_size)!=0:
            batch_data = np.zeros((len(t)%batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((len(t)%batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(len(t)%batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image=np.array(Image.fromarray((image * 255).astype(np.uint8)).resize((120, 120)).convert('RGB'))
                    batch_data[folder,idx,:,:,0] = (image[:,:,0])/255#normalise and feed in the image # divide by 255.0
                    batch_data[folder,idx,:,:,1] = (image[:,:,1])/255#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2])/255#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1 # OHE
            yield batch_data, batch_labels


In [16]:
input_shape = (21,120,120,3)

#write your model here
model_3 = Sequential()
model_3.add(Conv3D(64, (2,2,3), strides=(1,1,1), padding='same', input_shape=input_shape))
model_3.add(BatchNormalization())
model_3.add(Activation('relu'))
model_3.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,1)))

model_3.add(Conv3D(128, (2,2,3), strides=(1,1,1), padding='same', input_shape=input_shape))
model_3.add(BatchNormalization())
model_3.add(Activation('relu'))
model_3.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model_3.add(Conv3D(256, (2,2,3), strides=(1,1,1), padding='same', input_shape=input_shape))
model_3.add(BatchNormalization())
model_3.add(Activation('relu'))
model_3.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,1)))

model_3.add(Conv3D(512, (2,2,3), strides=(1,1,1), padding='same', input_shape=input_shape))
model_3.add(BatchNormalization())
model_3.add(Activation('relu'))
model_3.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,1)))

#Flatten Layers
model_3.add(Flatten())

model_3.add(Dense(1000, activation='relu'))
model_3.add(Dropout(0.5))

model_3.add(Dense(500, activation='relu'))
model_3.add(Dropout(0.5))

#softmax layer
model_3.add(Dense(5, activation='softmax'))

In [17]:
optimiser =  optimizers.Adam(lr=0.001) # write your optimizer
model_3.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_3.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_7 (Conv3D)            (None, 21, 120, 120, 64)  2368      
_________________________________________________________________
batch_normalization_7 (Batch (None, 21, 120, 120, 64)  256       
_________________________________________________________________
activation_7 (Activation)    (None, 21, 120, 120, 64)  0         
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 10, 60, 119, 64)   0         
_________________________________________________________________
conv3d_8 (Conv3D)            (None, 10, 60, 119, 128)  98432     
_________________________________________________________________
batch_normalization_8 (Batch (None, 10, 60, 119, 128)  512       
_________________________________________________________________
activation_8 (Activation)    (None, 10, 60, 119, 128) 

In [18]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [21]:
model_3.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=20, verbose=1, 
                    validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 20
Epoch 1/20
34/34 [==============================] - 65s 2s/step - loss: 164.4209 - categorical_accuracy: 0.2833 - val_loss: 5.3459 - val_categorical_accuracy: 0.2900
Epoch 2/20
34/34 [==============================] - 53s 2s/step - loss: 2.2921 - categorical_accuracy: 0.2668 - val_loss: 6.1360 - val_categorical_accuracy: 0.2300
Epoch 3/20
34/34 [==============================] - 53s 2s/step - loss: 1.4222 - categorical_accuracy: 0.3656 - val_loss: 2.1139 - val_categorical_accuracy: 0.2300
Epoch 4/20
34/34 [==============================] - 54s 2s/step - loss: 1.2363 - categorical_accuracy: 0.4492 - val_loss: 2.1521 - val_categorical_accuracy: 0.2300
Epoch 5/20
34/34 [==============================] - 53s 2s/step - loss: 1.1444 - categorical_accuracy: 0.4858 - val_loss: 2.5184 - val_categorical_accuracy: 0.2800
Epoch 6/20
34/34 [==============================] - 54s 2s/step - loss: 1.0101 - categorical_accuracy: 0.5613 - val_loss: 2.27

#### Comment:
The model is still not able to generalise well on the validation data

## Model 3 - Experiment Number 3

We will increase the batch_size from 20 to 40 and see the model performance

- When the batch size was increased from 20 to 40, we observed OOM error.
- On further reducing the batch_size, it was observed that 23 is the max batch_size which the GPU machine accepted.

In [19]:
train_generator = generator(train_path, train_doc, 23)
val_generator = generator(val_path, val_doc, 23)

In [20]:
model_3.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=20, verbose=1, 
                    validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 23
Epoch 1/20
34/34 [==============================] - 164s 4s/step - loss: 128.2598 - categorical_accuracy: 0.2630 - val_loss: 17.2761 - val_categorical_accuracy: 0.2100
Epoch 2/20
34/34 [==============================] - 63s 2s/step - loss: 1.8419 - categorical_accuracy: 0.3027 - val_loss: 1.6877 - val_categorical_accuracy: 0.2400
Epoch 3/20
34/34 [==============================] - 62s 2s/step - loss: 1.2224 - categorical_accuracy: 0.4677 - val_loss: 1.7947 - val_categorical_accuracy: 0.2700
Epoch 4/20
34/34 [==============================] - 62s 2s/step - loss: 1.1859 - categorical_accuracy: 0.4672 - val_loss: 1.7521 - val_categorical_accuracy: 0.1600
Epoch 5/20
34/34 [==============================] - 61s 2s/step - loss: 1.0079 - categorical_accuracy: 0.5871 - val_loss: 1.7296 - val_categorical_accuracy: 0.2600
Epoch 6/20
34/34 [==============================] - 63s 2s/step - loss: 0.9345 - categorical_accuracy: 0.6004 - val_loss: 1.

##### Comment:
Although, Accuracy on the train set has increased, the model is not performing well on validation set.

## Model 3 - Experiment Number 4

In this experiment, we will use SGD optimiser and observe the model's performance

In [21]:
optimiser = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7)
model_3.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_3.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_7 (Conv3D)            (None, 21, 120, 120, 64)  2368      
_________________________________________________________________
batch_normalization_7 (Batch (None, 21, 120, 120, 64)  256       
_________________________________________________________________
activation_7 (Activation)    (None, 21, 120, 120, 64)  0         
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 10, 60, 119, 64)   0         
_________________________________________________________________
conv3d_8 (Conv3D)            (None, 10, 60, 119, 128)  98432     
_________________________________________________________________
batch_normalization_8 (Batch (None, 10, 60, 119, 128)  512       
_________________________________________________________________
activation_8 (Activation)    (None, 10, 60, 119, 128) 

In [22]:
train_generator = generator(train_path, train_doc, 23)
val_generator = generator(val_path, val_doc, 23)

In [23]:
model_3.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=20, verbose=1, 
                    validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 23
Epoch 1/20
34/34 [==============================] - 64s 2s/step - loss: 0.1609 - categorical_accuracy: 0.9496 - val_loss: 7.7006 - val_categorical_accuracy: 0.2800
Epoch 2/20
34/34 [==============================] - 60s 2s/step - loss: 0.1752 - categorical_accuracy: 0.9245 - val_loss: 1.7093 - val_categorical_accuracy: 0.6800
Epoch 3/20
34/34 [==============================] - 62s 2s/step - loss: 0.1167 - categorical_accuracy: 0.9571 - val_loss: 1.2959 - val_categorical_accuracy: 0.7300
Epoch 4/20
34/34 [==============================] - 62s 2s/step - loss: 0.1015 - categorical_accuracy: 0.9596 - val_loss: 1.5252 - val_categorical_accuracy: 0.6500
Epoch 5/20
34/34 [==============================] - 60s 2s/step - loss: 0.1671 - categorical_accuracy: 0.9397 - val_loss: 4.6884 - val_categorical_accuracy: 0.3100
Epoch 6/20
34/34 [==============================] - 61s 2s/step - loss: 0.1575 - categorical_accuracy: 0.9488 - val_loss: 1.4310

###### Comment:

The model is overfitting. Next, we will try to change the number of images, filtersize and number of features

## Model 4 - Experiment Number 5

- Using dropout and number of epochs = 25
- Reducing number of images and changing the number of features in Conv3D layers

In [11]:
input_shape = (16,120,120,3)

#write your model here
model_4= Sequential()
model_4.add(Conv3D(16, (2,2,3), strides=(1,1,1), padding='same', input_shape=input_shape))
model_4.add(BatchNormalization())
model_4.add(Activation('relu'))
model_4.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,1)))

model_4.add(Conv3D(32, (2,2,3), strides=(1,1,1), padding='same', input_shape=input_shape))
model_4.add(BatchNormalization())
model_4.add(Activation('relu'))
model_4.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model_4.add(Conv3D(64, (2,2,3), strides=(1,1,1), padding='same', input_shape=input_shape))
model_4.add(BatchNormalization())
model_4.add(Activation('relu'))
model_4.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,1)))

model_4.add(Conv3D(128, (2,2,3), strides=(1,1,1), padding='same', input_shape=input_shape))
model_4.add(BatchNormalization())
model_4.add(Activation('relu'))
model_4.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,1)))
model_4.add(Dropout(0.25))

#Flatten Layers
model_4.add(Flatten())

model_4.add(Dense(256, activation='relu'))
model_4.add(Dropout(0.25))

model_4.add(Dense(128, activation='relu'))
model_4.add(Dropout(0.25))

#softmax layer
model_4.add(Dense(5, activation='softmax'))

In [12]:
optimiser =  optimizers.Adam(lr=0.001) # write your optimizer
model_4.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_4.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_3 (Conv3D)            (None, 16, 120, 120, 16)  592       
_________________________________________________________________
batch_normalization_3 (Batch (None, 16, 120, 120, 16)  64        
_________________________________________________________________
activation_3 (Activation)    (None, 16, 120, 120, 16)  0         
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 8, 60, 119, 16)    0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 8, 60, 119, 32)    6176      
_________________________________________________________________
batch_normalization_4 (Batch (None, 8, 60, 119, 32)    128       
_________________________________________________________________
activation_4 (Activation)    (None, 8, 60, 119, 32)   

In [13]:
train_generator = generator(train_path, train_doc, 20)
val_generator = generator(val_path, val_doc, 20)

In [14]:
model_4.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs= 25, verbose=1, 
                    validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 20
Epoch 1/25
34/34 [==============================] - 135s 4s/step - loss: 18.3574 - categorical_accuracy: 0.2638 - val_loss: 2.1463 - val_categorical_accuracy: 0.1700
Epoch 2/25
34/34 [==============================] - 44s 1s/step - loss: 2.8783 - categorical_accuracy: 0.3841 - val_loss: 2.3272 - val_categorical_accuracy: 0.1300
Epoch 3/25
34/34 [==============================] - 43s 1s/step - loss: 1.7994 - categorical_accuracy: 0.3075 - val_loss: 1.9584 - val_categorical_accuracy: 0.1600
Epoch 4/25
34/34 [==============================] - 44s 1s/step - loss: 1.3046 - categorical_accuracy: 0.4288 - val_loss: 2.5706 - val_categorical_accuracy: 0.2100
Epoch 5/25
34/34 [==============================] - 45s 1s/step - loss: 1.1352 - categorical_accuracy: 0.5124 - val_loss: 2.4604 - val_categorical_accuracy: 0.2000
Epoch 6/25
34/34 [==============================] - 44s 1s/step - loss: 1.2488 - categorical_accuracy: 0.4263 - val_loss: 2.40

##### Comment:

After few experiments, we observe that the model is still not able to generalize well on the validation set. Let's next use **Transfer Learning+RNN**


##  Using transfer learning+RNN

## Model 5 - VGG16 + GRU

In [4]:
x = 21 # number of frames
y = 120 # image width
z = 120 # image height

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    # It is not possible to work with all the 30 images, as it will take too long processing time.
    # So lets choose randomly 21 images, as this is more computationally expensive
    img_idx = [0,1,2,3,4,6,10,12,13,14,16,17,18,20,21,22,24,25,27,28,29] #create a list of image numbers you want to use for a particular video(incase if u want to try with lesser images)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,21,120,120,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image=np.array(Image.fromarray((image * 255).astype(np.uint8)).resize((120, 120)).convert('RGB'))
                    
                    batch_data[folder,idx,:,:,0] = (image[:,:,0])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2])/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1 # OHE
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if(len(t)%batch_size)!=0:
            batch_data = np.zeros((len(t)%batch_size,21,120,120,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((len(t)%batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(len(t)%batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image=np.array(Image.fromarray((image * 255).astype(np.uint8)).resize((120, 120)).convert('RGB'))
                    
                    batch_data[folder,idx,:,:,0] = (image[:,:,0])/255
                    batch_data[folder,idx,:,:,1] = (image[:,:,1])/255
                    batch_data[folder,idx,:,:,2] = (image[:,:,2])/255
                    
                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1 # OHE
            yield batch_data, batch_labels

In [5]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


In [6]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout,GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras import layers 
import warnings
warnings.filterwarnings('ignore')

In [7]:
base_model = VGG16(include_top=False, weights = 'imagenet', input_shape = (120,120,3))
for layer in base_model.layers:
    layer.trainable = False
    
x = base_model.output
x = Flatten()(x)
x = layers.Dropout(0.5)(x)
features = Dense(64, activation='relu')(x)
conv_model = Model(inputs= base_model.input, outputs=features)

In [9]:
cnn_rnn_model = Sequential()
cnn_rnn_model.add(TimeDistributed(conv_model, input_shape=(21,120,120,3)))
cnn_rnn_model.add(TimeDistributed(BatchNormalization()))
cnn_rnn_model.add(TimeDistributed(Flatten()))


cnn_rnn_model.add(GRU(256, activation = 'relu', return_sequences=True))
cnn_rnn_model.add(GRU(64))
cnn_rnn_model.add(Dropout(0.5))
cnn_rnn_model.add(Dense(8, activation='relu'))
cnn_rnn_model.add(Dense(5, activation='softmax'))

In [10]:
optimiser =  optimizers.Adam(lr=0.001) # write your optimizer
cnn_rnn_model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (cnn_rnn_model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_3 (TimeDist (None, 21, 64)            15009664  
_________________________________________________________________
time_distributed_4 (TimeDist (None, 21, 64)            256       
_________________________________________________________________
time_distributed_5 (TimeDist (None, 21, 64)            0         
_________________________________________________________________
gru (GRU)                    (None, 21, 256)           247296    
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                61824     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                

In [11]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [12]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_weights_only=False, mode='auto', save_freq='epoch')

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [13]:
# if #images=55, batch_size=5, then we have 11 batches and 11 epochs
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [14]:
cnn_rnn_model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 20
Epoch 1/30
34/34 [==============================] - 160s 4s/step - loss: 1.5746 - categorical_accuracy: 0.2701 - val_loss: 1.5693 - val_categorical_accuracy: 0.2600
Epoch 2/30
34/34 [==============================] - 50s 2s/step - loss: 1.0972 - categorical_accuracy: 0.5352 - val_loss: 1.2816 - val_categorical_accuracy: 0.5100
Epoch 3/30
34/34 [==============================] - 49s 1s/step - loss: 0.8782 - categorical_accuracy: 0.6429 - val_loss: 0.9376 - val_categorical_accuracy: 0.6200
Epoch 4/30
34/34 [==============================] - 49s 1s/step - loss: 0.5337 - categorical_accuracy: 0.8185 - val_loss: 1.5166 - val_categorical_accuracy: 0.4700
Epoch 5/30
34/34 [==============================] - 49s 1s/step - loss: 0.4047 - categorical_accuracy: 0.8778 - val_loss: 1.2352 - val_categorical_accuracy: 0.5900
Epoch 6/30
34/34 [==============================] - 49s 1s/step - loss: 0.3226 - categorical_accuracy: 0.9135 - val_loss: 1.049

##### Comment:
This model is overfitting too. We will next try to use MobileNet which reduces the number of parameters significantly and thus results in lightweight network. MobileNet can train our classifiers faster.

## Final Model -  Model 6 :  MobileNet + GRU

In [4]:
x = 21 # number of frames
y = 120 # image width
z = 120 # image height

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    # It is not possible to work with all the 30 images, as it will take too long processing time.
    # So lets choose randomly 21 images, as this is more computationally expensive
    img_idx = [0,1,2,3,4,6,10,12,13,14,16,17,18,20,21,22,24,25,27,28,29] #create a list of image numbers you want to use for a particular video(incase if u want to try with lesser images)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,21,120,120,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image=np.array(Image.fromarray((image * 255).astype(np.uint8)).resize((120, 120)).convert('RGB'))
                    
                    batch_data[folder,idx,:,:,0] = (image[:,:,0])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2])/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1 # OHE
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if(len(t)%batch_size)!=0:
            batch_data = np.zeros((len(t)%batch_size,21,120,120,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((len(t)%batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(len(t)%batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image=np.array(Image.fromarray((image * 255).astype(np.uint8)).resize((120, 120)).convert('RGB'))
                    
                    batch_data[folder,idx,:,:,0] = (image[:,:,0])/255
                    batch_data[folder,idx,:,:,1] = (image[:,:,1])/255
                    batch_data[folder,idx,:,:,2] = (image[:,:,2])/255
                    
                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1 # OHE
            yield batch_data, batch_labels

In [5]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


In [6]:
from tensorflow.keras.applications import mobilenet
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout,GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras import layers 

In [7]:
mobilenet_transfer = mobilenet.MobileNet(weights='imagenet', include_top=False)
mob_model = Sequential()
mob_model.add(TimeDistributed(mobilenet_transfer,input_shape=(21,120,120,3)))

mob_model.add(TimeDistributed(BatchNormalization()))
mob_model.add(TimeDistributed(MaxPooling2D((2, 2))))
mob_model.add(TimeDistributed(Flatten()))

mob_model.add(GRU(64))
mob_model.add(Dropout(0.3))

mob_model.add(Dense(32,activation='relu'))
mob_model.add(Dropout(0.3))

mob_model.add(Dense(5, activation='softmax'))

In [8]:
optimiser =  optimizers.Adam() # write your optimizer
mob_model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (mob_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 21, 3, 3, 1024)    3228864   
_________________________________________________________________
time_distributed_1 (TimeDist (None, 21, 3, 3, 1024)    4096      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 21, 1, 1, 1024)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 21, 1024)          0         
_________________________________________________________________
gru (GRU)                    (None, 64)                209280    
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                2

In [9]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [10]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1,save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [11]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [12]:
mob_model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs= num_epochs, verbose=1, 
                    callbacks=callbacks_list,validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 20
Epoch 1/20
34/34 [==============================] - 195s 5s/step - loss: 1.6695 - categorical_accuracy: 0.3135 - val_loss: 1.3592 - val_categorical_accuracy: 0.4700

Epoch 00001: val_loss improved from inf to 1.35924, saving model to model_init_2021-10-2113_35_59.183434/model-00001-1.50738-0.38009-1.35924-0.47000.h5
Epoch 2/20
34/34 [==============================] - 53s 2s/step - loss: 1.0552 - categorical_accuracy: 0.5871 - val_loss: 1.0167 - val_categorical_accuracy: 0.5600

Epoch 00002: val_loss improved from 1.35924 to 1.01671, saving model to model_init_2021-10-2113_35_59.183434/model-00002-0.97368-0.61538-1.01671-0.56000.h5
Epoch 3/20
34/34 [==============================] - 52s 2s/step - loss: 0.5491 - categorical_accuracy: 0.8027 - val_loss: 0.4173 - val_categorical_accuracy: 0.8600

Epoch 00003: val_loss improved from 1.01671 to 0.41732, saving model to model_init_2021-10-2113_35_59.183434/model-00003-0.53838-0.81146-0.41732

##### Comment:
This model is performing well on both training and validation data.

## Note:

In the final model, we have saved only best epochs by setting save_best_only = True due to space issues/constraints on nimblebox.ai

### Summary of the Final Model

**Accuracy**: 
- Train =  99% 
- Validation = 94%

**loss**:
- Train_loss = 5%
- Val_loss = 21%

**Number of parameters** in the final model has reduced considerably and it is as follows:

- Total params: 3,444,485
- Trainable params: 3,420,549
- Non-trainable params: 23,936

### Conclusion

The first CNN3D model which we have created performed poorly on both train and val data. Thereafter, following models performance increased by experimenting with few parameters. In this assignment, we learnt to play around with CNN models by changing number of frames, epochs, learning rate(optimiser's), dropouts(increasing and decreasing) and use Transfer Learning.

One issue which persisted through various experiments that model was not able to generalise well on the validation set. In order to overcome overfitting, we have included batch normalisation, used dropouts and tweaked learning rate by using ReduceLROnPlateau.